# Load Data

In [ ]:
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xvf  'dakshina_dataset_v1.0.tar'

--2021-05-25 15:35:27--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G  42.8MB/s    in 18s     

2021-05-25 15:35:45 (106 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]

dakshina_dataset_v1.0/bn/
dakshina_dataset_v1.0/bn/lexicons/
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.test.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.train.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.dev.tsv
dakshina_dataset_v1.0/bn/native_script_wikipedia/
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.valid.text.shuf.txt.gz

# load and create model

In [ ]:
%pip install wandb -q
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 1.8MB 7.3MB/s 
     |████████████████████████████████| 174kB 48.1MB/s 
     |████████████████████████████████| 102kB 11.7MB/s 
     |████████████████████████████████| 133kB 47.6MB/s 
     |████████████████████████████████| 71kB 9.1MB/s 


In [ ]:
input_token_index = None
target_token_index = None
MAX_LEN_input = None
MAX_LEN_target = None
num_encoder_tokens = 30
num_decoder_tokens = 70
input_tokenizer = None
target_tokenizer = None

In [ ]:
def tokenize(data,vocab_size):
  tokenizer = Tokenizer(num_words=vocab_size,char_level=True)
  tokenizer.fit_on_texts(data)
  temp=tokenizer.texts_to_sequences(data)
  # print(data[:3])
  # print(temp[:3])
  dictionary = tokenizer.word_index
  return temp , dictionary , tokenizer

In [ ]:
def load_and_preprocess():
  global input_token_index , target_token_index , MAX_LEN_input , MAX_LEN_target ,num_decoder_tokens,num_encoder_tokens , input_tokenizer , target_tokenizer
  data_path = 'dakshina_dataset_v1.0/gu/lexicons/gu.translit.sampled.train.tsv'
  input_texts = []
  target_texts = []
  with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
  for line in lines[:-1]:
    temp = line.split('\t')
    input_text, target_text = temp[1],temp[0] 
    target_text = "\t" + target_text + "\n"
    input_text = input_text+"\n"

    input_texts.append(input_text)
    target_texts.append(target_text)
  
  MAX_LEN_input = max([len(txt) for txt in input_texts])
  MAX_LEN_target = max([len(txt) for txt in target_texts])

  # tokenize
  encoder_input , input_token_index , input_tokenizer = tokenize(input_texts , num_encoder_tokens)
  decoder_input , target_token_index, target_tokenizer = tokenize(target_texts , num_decoder_tokens) 

  # padding
  encoder_input_data = pad_sequences(encoder_input, maxlen=MAX_LEN_input, dtype='int32', padding='post', truncating='post',value= input_token_index["\n"])
  decoder_input_data = pad_sequences(decoder_input, maxlen=MAX_LEN_target, dtype='int32', padding='post', truncating='post',value=target_token_index["\n"])

  decoder_target_data = np.zeros((len(input_texts), MAX_LEN_target, num_decoder_tokens), dtype="float32")
  for i,  target_text in enumerate(target_texts):
    for t, char in enumerate(target_text):
      if t > 0:
        decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_target_data[i, t:, target_token_index["\n"]] = 1.0

  return encoder_input_data , decoder_input_data, decoder_target_data



In [ ]:
def load_val_data(data_path = 'dakshina_dataset_v1.0/gu/lexicons/gu.translit.sampled.dev.tsv'):
  global input_token_index , target_token_index , MAX_LEN_input , MAX_LEN_target ,num_decoder_tokens,num_encoder_tokens , input_tokenizer , target_tokenizer
  
  input_texts = []
  target_texts = []
  with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
  for line in lines[:-1]:
    temp = line.split('\t')
    target_text , input_text = temp[0],temp[1] 
    target_text = "\t" + target_text + "\n"
    input_text = input_text+"\n"

    input_texts.append(input_text)
    target_texts.append(target_text)
  
  # tokenize
  encoder_input  = input_tokenizer.texts_to_sequences(input_texts)
  decoder_input  = target_tokenizer.texts_to_sequences(target_texts) 

  # padding
  encoder_input_data = pad_sequences(encoder_input, maxlen=MAX_LEN_input, dtype='int32', padding='post', truncating='post',value= input_token_index["\n"])
  decoder_input_data = pad_sequences(decoder_input, maxlen=MAX_LEN_target, dtype='int32', padding='post', truncating='post',value=target_token_index["\n"])

  decoder_target_data = np.zeros((len(input_texts), MAX_LEN_target, num_decoder_tokens), dtype="float32")
  for i,  target_text in enumerate(target_texts):
    for t, char in enumerate(target_text):
      if char == 'ૠ':
        char = 'ઋ'
      if t > 0:
        decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_target_data[i, t:, target_token_index["\n"]] = 1.0

  return encoder_input_data , decoder_input_data, decoder_target_data

# load_val_data()

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Dense,LSTM,Input,GRU,SimpleRNN,Dropout,Embedding

In [ ]:
def create_model(m_name="LSTM",n_e_layers=1,n_d_layers=1,latent_dim = 100,embedding_size = 16,dropout = 0 , recurrent_dropout = 0):
  keras.backend.clear_session()
  # num_encoder_tokens = 30
  # num_decoder_tokens = 50
  # Define an input sequence and process it.
  input1 = Input(shape=(None,),name= "input_1")
  encoder_inputs = Embedding(input_dim = num_encoder_tokens, output_dim = embedding_size)(input1)

  encoder = globals()[m_name](latent_dim,dropout=dropout,recurrent_dropout = recurrent_dropout, return_state=True,return_sequences=True)
  e_o = encoder(encoder_inputs)
  prev = e_o
  for i in range(1,n_e_layers):
    e = globals()[m_name](latent_dim, dropout=dropout,recurrent_dropout = recurrent_dropout,return_state=True,return_sequences=True)
    e_o = e(prev[0])
    prev = e_o
  
  input2 = Input(shape=(None,),name="input_2")
  decoder_inputs = Embedding(input_dim = num_decoder_tokens, output_dim = embedding_size)(input2)
  d_l = globals()[m_name](latent_dim,dropout=dropout,recurrent_dropout = recurrent_dropout, return_sequences=True, return_state=True)(decoder_inputs,initial_state = e_o[1:])
  p_d = d_l[0]
  for i in range(1,n_d_layers):
    d_l = globals()[m_name](latent_dim,dropout=dropout,recurrent_dropout = recurrent_dropout, return_state=True, return_sequences=True)(p_d,initial_state = e_o[1:])
    p_d = d_l[0]

  
  decoder_dense = Dense(num_decoder_tokens, activation="softmax")
  decoder_outputs = decoder_dense(d_l[0])

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([input1,input2], decoder_outputs)

  return model


# sweep

In [ ]:
def train():
  run = wandb.init()
  c = run.config
  name = "model_"+c.model+"_o_"+c.optimizer+"_el_"+str(c.encoder_layers)+"_dl_"+str(c.decoder_layers)+"_hs_"+str(c.hidden_size)+"_em_"+str(c.embedding_size)+"_d_"+str(c.dropout)
  run.name = name
  print(name)
  batch_size = 128
  epochs = 20

  encoder_input_data,decoder_input_data ,decoder_target_data = load_and_preprocess()
  val_encoder_input_data,val_decoder_input_data ,val_decoder_target_data = load_val_data()
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

  model = create_model(c.model,c.encoder_layers,c.decoder_layers,c.hidden_size,c.embedding_size,c.dropout,0)
  model.compile(optimizer=c.optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
  model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=([val_encoder_input_data, val_decoder_input_data],val_decoder_target_data),
    # validation_split=0.2,
    callbacks=[WandbCallback(),es]
  )
  temp = model.predict([val_encoder_input_data, val_decoder_input_data]).argmax(axis=-1)
  val_word_acc = sum((temp[:,:-1] == val_decoder_input_data[:,1:]).all(axis=-1)) / len(val_encoder_input_data)
  # print(val_word_acc)

  temp = model.predict([encoder_input_data,decoder_input_data]).argmax(axis=-1)
  train_word_acc = sum((temp[:,:-1] == decoder_input_data[:,1:]).all(axis=-1)) / len(encoder_input_data)
  # print(train_word_acc)

  wandb.log({"val_word_acc" : round(val_word_acc,4) , "train_word_acc" : round(train_word_acc,4)})
  return

In [ ]:
sweep_config={
    'method' : 'random' ,
    'metric' : { 'name' : 'val_word_acc' , 'goal' : 'maximize' } ,
    'parameters' : {
        'model' : { 'values' : ['LSTM','GRU','SimpleRNN'] },
        'dropout' : { 'values' : [0.1,0.2]},
        'embedding_size' : {'values' : [64,128]},
        'encoder_layers' : {'values' : [1,2,3]},
        'decoder_layers' : {'values' : [1,2,3]},
        'hidden_size' : {'values' : [128,256,512]},
        'optimizer' : {'values' : ['rmsprop' ,'adam']}
    }
}


In [ ]:

sweepid = wandb.sweep(sweep_config,project="DL_A3_Q2_final",entity ="sonagara")
wandb.agent(sweepid,train)

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: u6pg8nh7
Sweep URL: https://wandb.ai/sonagara/DL_A3_Q1_testing1/sweeps/u6pg8nh7


...

epoch,0
loss,1.1351
accuracy,0.70639
val_loss,0.92348
val_accuracy,0.74362
_runtime,135
_timestamp,1621252192
_step,1
best_val_loss,0.92348
best_epoch,0
val_word_acc,0.0001


epoch,▁
loss,▁
accuracy,▁
val_loss,▁
val_accuracy,▁
_runtime,▁█
_timestamp,▁█
_step,▁█
val_word_acc,▁
train_word_acc,▁


wandb: Agent Starting Run: c5yx5vew with config:
wandb: 	decoder_layers: 1
wandb: 	dropout: 0.1
wandb: 	embedding_size: 128
wandb: 	encoder_layers: 2
wandb: 	hidden_size: 128
wandb: 	model: LSTM
wandb: 	optimizer: rmsprop


model_LSTM_o_rmsprop_el_2_dl_1_hs_128_em_128_d_0.1
208/823 [======>.......................] - ETA: 22s - loss: 1.4987 - accuracy: 0.6681

wandb: Ctrl + C detected. Stopping sweep.


# best model

In [ ]:
def train_best():
  m_name = "GRU"
  encoder_layers = 3
  decoder_layers = 3
  latent_dim = 128
  embedding_size = 128
  dropout = 0.1
  batch_size = 128
  recurrent_dropout = 0  # 0 to use cudnnlstm which is faster than lstm
  optimizer = "adam"
  encoder_input_data,decoder_input_data ,decoder_target_data = load_and_preprocess()
  val_encoder_input_data,val_decoder_input_data ,val_decoder_target_data = load_val_data()
  

  model = create_model(m_name,encoder_layers,decoder_layers,latent_dim,embedding_size,dropout,recurrent_dropout)
  # model = keras.models.load_model("s2s")
  model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

  model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=20,
    # validation_split = 0.2,
    validation_data=([val_encoder_input_data, val_decoder_input_data],val_decoder_target_data),
    callbacks=[es]
  )

  temp = model.predict([val_encoder_input_data, val_decoder_input_data]).argmax(axis=-1)
  val_word_acc = sum((temp[:,:-1] == val_decoder_input_data[:,1:]).all(axis=-1)) / len(val_encoder_input_data)
  print(val_word_acc)

  temp = model.predict([encoder_input_data,decoder_input_data]).argmax(axis=-1)
  train_word_acc = sum((temp[:,:-1] == decoder_input_data[:,1:]).all(axis=-1)) / len(encoder_input_data)
  print(train_word_acc)
  


  # Save model
  model.save("s2s")
  # print(test_acc("LSTM",100,1,2))
train_best()

Epoch 1/20
823/823 [==============================] - 57s 23ms/step - loss: 1.3186 - accuracy: 0.6925 - val_loss: 0.8526 - val_accuracy: 0.7584
Epoch 2/20
823/823 [==============================] - 17s 21ms/step - loss: 0.8142 - accuracy: 0.7657 - val_loss: 0.6200 - val_accuracy: 0.8201
Epoch 3/20
823/823 [==============================] - 17s 20ms/step - loss: 0.5997 - accuracy: 0.8219 - val_loss: 0.4447 - val_accuracy: 0.8639
Epoch 4/20
823/823 [==============================] - 17s 20ms/step - loss: 0.4130 - accuracy: 0.8736 - val_loss: 0.3071 - val_accuracy: 0.9040
Epoch 5/20
823/823 [==============================] - 17s 21ms/step - loss: 0.2856 - accuracy: 0.9109 - val_loss: 0.2438 - val_accuracy: 0.9237
Epoch 6/20
823/823 [==============================] - 17s 21ms/step - loss: 0.2178 - accuracy: 0.9314 - val_loss: 0.2115 - val_accuracy: 0.9335
Epoch 7/20
823/823 [==============================] - 17s 21ms/step - loss: 0.1795 - accuracy: 0.9433 - val_loss: 0.1972 - val_accuracy:

INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


In [ ]:
import re
def load_test_data():
  global input_token_index , MAX_LEN_input , num_encoder_tokens , input_tokenizer
  data_path = 'dakshina_dataset_v1.0/gu/lexicons/gu.translit.sampled.test.tsv'
  input_texts=[]
  target_texts = []
  s = r'ૠ'
  # input_characters = set()
  with open(data_path,"r",encoding="utf-8") as f:
    lines = f.read().split("\n")
  for line in lines[:-1]:
    temp = line.split("\t")
    input_text , target_text = temp[1] ,temp[0]
    target_text = re.sub(s,'ઋ',target_text)
    input_text = input_text + "\n"
    target_text = target_text + "\n"
    
    input_texts.append(input_text)
    target_texts.append(target_text)
  
  encoder_input = input_tokenizer.texts_to_sequences(input_texts)
  encoder_input_data = pad_sequences(encoder_input, maxlen=MAX_LEN_input, dtype='int32', padding='post', truncating='post',value= input_token_index["\n"])
  print(encoder_input_data.shape)
  
  return encoder_input_data , input_texts ,target_texts
# load_test_data()

In [ ]:
def enc_dec(m_name="LSTM" ,latent_dim = 100, n_e_layers = 1,n_d_layers = 1):
  # keras.backend.clear_session()
  model = keras.models.load_model("s2s")
  # model.summary()
 
  if (n_e_layers == 1):
    l_name = ""
  else:
    l_name = "_"+str(n_e_layers-1)

  if (m_name == "SimpleRNN"):
    n_name = "simple_rnn"
  else:
    n_name = m_name

  # encoder
  encoder_inputs = model.input[0]
  encoder_outputs, *encoder_states = model.get_layer(n_name.lower()+l_name).output  # last encoding layer
  encoder_model = keras.Model(encoder_inputs, encoder_states)
  # encoder_model.summary()

  # decoder
  decoder_inputs = model.input[1] 
  decoder_embed = model.get_layer("embedding_1")(decoder_inputs)
  
  decoder_states_inputs = []
  decoder_states_inputs.append(keras.Input(shape=(latent_dim,)))
  if (m_name == "LSTM"):
    decoder_states_inputs.append(keras.Input(shape=(latent_dim,)))
  
  d_o = model.get_layer(n_name.lower()+"_"+str(n_e_layers))(decoder_embed, initial_state=decoder_states_inputs)
  for i in range(1+n_e_layers,n_d_layers+n_e_layers):
    d_o = model.get_layer(n_name.lower()+"_"+str(i))(d_o[0], initial_state=decoder_states_inputs)
  decoder_outputs = d_o[0]
  decoder_states = d_o[1:]
  decoder_dense = model.get_layer("dense")
  decoder_outputs = decoder_dense(decoder_outputs)
  decoder_model = keras.Model([decoder_inputs ,  decoder_states_inputs], [decoder_outputs] + decoder_states)
  # decoder_model.summary()

  return encoder_model , decoder_model
# enc_dec("LSTM",256,3,2)


In [ ]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq,encoder_model,decoder_model):
    global num_decoder_tokens , target_token_index , reverse_target_char_index , MAX_LEN_target
    # Encode the input as state vectors.

    states_value = encoder_model.predict(input_seq)
    # print(states_value)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index["\t"]
    # print(target_seq)
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, *states = decoder_model.predict([target_seq, states_value])
        # print(len(states))
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > MAX_LEN_target:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = target_token_index[sampled_char]

        # Update states
        states_value = states
    return decoded_sentence


In [ ]:
decode_results = []
test_input = None
def test_acc(m_name="LSTM" ,latent_dim = 100, n_e_layers = 1,n_d_layers = 1):
  global decode_results,test_input
  x_test ,x ,y_test = load_test_data()
  test_input = x
  enc ,dec = enc_dec(m_name , latent_dim , n_e_layers ,n_d_layers)
  score = 0 
  # print(len(y_test))

  for seq_index in range(len(y_test)):
    
    input_seq = x_test[seq_index : seq_index + 1]
    print(x[seq_index])
    print(input_seq)
    decoded_sentence = decode_sequence(input_seq,enc,dec)
    decode_results.append(decoded_sentence)
    if (y_test[seq_index] == decoded_sentence):
      score += 1
    # print("-")
    # print("original sentence:", y_test[seq_index])
    # print("Decoded sentence:", decoded_sentence)
  print(score/len(y_test))
  return score/len(y_test)

In [ ]:
print(test_acc("GRU",128,3,3)) #0.32623156271088405

In [ ]:
def save_csv():
  global test_input , decode_results
  dict = {"Input Sentence" : test_input , "Decoded Sentence" : decode_results}
  df = pd.DataFrame(dict)
  df.to_csv("predictions_vanilla.csv")
save_csv()

In [ ]:
def grid():
  global test_input,decode_results
  _,_,y = load_test_data()
  for i in range(30,40):
    if (y[i] == decode_results[i]):
      print("\033[1;32;47m"+test_input[i] +"\t"+ "\033[1;32;47m"+ decode_results[i])
    else:
      print("\033[1;31;47m" +test_input[i]+ "\t"+"\033[1;31;47m"+ decode_results[i])

grid()

(10373, 23)
angrejimaan
	અંગ્રેજોમાં

angrejiman
	અંગ્રેજોમાં

anjaaramaa
	અંજરરાાં

anjaarma
	અંજરરમાં

anjaarmaan
	અંજારમાં

anjaarman
	અંજરરમાં

anjarma
	અંજરરમાં

anjarmaan
	અંજરરમાં

anjarman
	અંજરરમાં

andhaapo
	અંધાપો



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model = keras.models.load_model("s2s")
model.save("drive/My Drive/s2s")

INFO:tensorflow:Assets written to: drive/My Drive/s2s/assets


INFO:tensorflow:Assets written to: drive/My Drive/s2s/assets
